In [15]:
# instruciton datasetを今回のSFT形式に変換する

In [16]:
import os

#dataフォルダ内をリセット
os.system("mkdir data")
os.system("rm -rf data/*")

mkdir: cannot create directory ‘data’: File exists


0

In [17]:
from datasets import load_dataset
import json

# 自動生成

In [18]:
dataset=load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA",split="train")

In [19]:
print(len(dataset))

165671


In [20]:
def clean_autogen(text):
    if text is None:
        return ""
    text=text.strip()
    return text

In [21]:
"""
本番はこっち｡
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい
"""

#original template
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n### 応答:\n"

#custom template
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
answer_template="<SEP>応答<SEP>"

In [22]:
dataset[0]

{'question': '次の本を読んだことがある人物について、進化論や生物学に関する知識があるかどうか考察してください。『のとヒトの進化』',
 'database': 'hatakeyama-llm-team/WikiBookJa',
 'answer': ' まず、「の問題とヒトの進化」を読んだことがある人物については、それ自体が進化論や生物学に関する知識を持っている可能性が高いでしょう。この本は、ヒトの進化的起源についての深い分析と考察が含まれています。\n\n次に、進化論や生物学に関する知識を持っているかどうかを判断するためには、以下の点に注意してください。\n\n1. 進化論の基本概念: 人物が進化論の基礎的な概念や側面を理解しているかどうかを確認します。これには、遺伝子や突然変異、自然選択などが含まれます。\n2. 進化論の歴史: 人物が進化論の歴史的展開と重要な奮闘者について知っているかどうかを確認します。これには、ダーウィンやラマルクなどの名前が含まれます。\n3. 生物学的分類: 人物が生物学的分類法について理解しており、様々な生物種の進化的関係を把握できるかどうかを確認します。\n4. 現代進化論: 人物が最新の研究成果や分野内の議論について知っているかどうかを確認します。これには、Epigeneticsなどの新たな理解や進展が含まれます。\n5. 実際の例: 人物が進化論や生物学についての知識を実際の状況や現象に適用できるかどうかを確認します。これには、自然界で見られる様々な形式や行動が含まれます。\n\n上記の要素を踏まえて、「の問題とヒトの進化」を読んだことがある人物について、進化論や生物学に関する知識があるかどうかを調査します。',
 'score': -2.0}

In [23]:
records=[]

output_path=f"data/gousei.jsonl"
with open(output_path,"w") as f:
    f.write("")

for original_record in iter(dataset):
    q=clean_autogen(original_record["question"])
    a=clean_autogen(original_record["answer"])
    
    text=f"{question_template}{q}{answer_template}{a}"
    if a!="":
        records.append(text)



len(records)

165670

In [24]:
ds2=load_dataset("hatakeyama-llm-team/BumpoRikai",split="train")
ds2

Dataset({
    features: ['instruction', 'question', 'answer'],
    num_rows: 26448
})

In [25]:
for original_record in iter(ds2):
    q=clean_autogen(original_record["question"])
    a=clean_autogen(original_record["answer"])
    inst=clean_autogen(original_record["instruction"])
    
    text=f"{question_template}{inst}\n{q}{answer_template}{a}"

    records.append(text)
print(text)

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>次の質問への回答を生成。回答のみを記載してください。
問題　樹液からはチューインガムの原料が採取できる、中南米原産のアカテツ科の植物は何でしょう?<SEP>応答<SEP>サポジラ


In [33]:
import random
random.shuffle(records)
n_eval=500
n_train=30000

lines=[json.dumps({"text":text},ensure_ascii=False) for text in records]
with open (output_path,"w") as f:
    temp_lines=lines[:-n_eval]
    f.writelines(temp_lines[:n_train])

lines=[json.dumps({"text":text},ensure_ascii=False) for text in records]
with open (output_path+".eval","w") as f:
    f.writelines(lines[-n_eval:])

In [34]:
len(records)

192118